In [1]:
import glob
import sep
import astropy.units as u
import astropy.coordinates as coord
import matplotlib.pyplot as plt
import numpy as np
import astroalign as aa
from astropy.io import fits
from astroquery.ipac.irsa import Irsa
from astroquery.sdss import SDSS
from astropy.wcs import WCS
from astropy.wcs.utils import pixel_to_skycoord
from astropy.coordinates import SkyCoord
from matplotlib.colors import LogNorm, TABLEAU_COLORS
from photutils.aperture import aperture_photometry, CircularAperture, CircularAnnulus, ApertureStats
from regions import CirclePixelRegion, PixCoord
from photutils import centroids
import os

#imports
#FOR ASTROQUERRY: If you encounter an error related to "fields" in "SDSS.querry_crossid", try installing the newest development version of astroquerry rather than the default. There is a bug in the older official release version.

In [62]:
class FrameData:

    def extract_data(self, file):
        hdul = fits.open(file)
        image = hdul[1].data
        header = hdul[1].header
        return image, header

    def add_realigned_image(self, aligned_image):
        self.aligned_image = aligned_image

    def add_reference_star(self, icrs_coords, pixel_coords, x_min, x_max, SDSS_filter_band):
        new_star = Star(icrs_coords, pixel_coords, x_min, x_max, SDSS_filter_band)
        self.star_list.append(new_star)

    def __init__(self, file):
        self.star_list = []
        self.image, self.header = self.extract_data(file)


class FrameSet:

    def align_frames(self):
        first_image = self.image_list[0]
        try:
            aligned = [aa.register(image, first_image)[0] for image in self.image_list[0:]]
        except:
            aligned = self.image_list
            raise UserWarning("AstroAlign did not align the frames")
        self.aligned = aligned

    def store_aligned_frames(self):
        for frame_index in range(len(self.frame_list)):
            self.frame_list[frame_index].add_realigned_image(self.aligned[frame_index])

    def create_template(self):
        template = np.median(self.aligned, axis=0)
        self.template = template

    def find_sources_from_template(self):
        background_object = sep.Background(self.template)
        self.sep_template_sources = sep.extract(self.template - background_object.back(), 3*background_object.globalrms, minarea=25, segmentation_map=False)

    def create_reference_stars(self):
        for frame_index in range(len(self.frame_list)):
            for source in self.sep_template_sources:
                pixel_coords = [source['x'], source['y']]
                icrs_coords = pixel_to_skycoord(pixel_coords[0], pixel_coords[1], self.WCS_matrix).transform_to('icrs')
                x_min = source['xmin']
                x_max = source['xmax']
                self.frame_list[frame_index].add_reference_star(icrs_coords=icrs_coords, pixel_coords=pixel_coords, x_min=x_min, x_max=x_max, SDSS_filter_band=self.filter_band)

    def __init__(self, file_path):
        self.frame_list = []
        self.image_list = []
        self.header_list = []

        for file in file_path:
            frame = FrameData(file) #Create a new instance of FrameData for every file in the set directory
            self.frame_list.append(frame)
            self.image_list.append(frame.image)
            self.header_list.append(frame.header)

        self.WCS_matrix = WCS(self.header_list[0]) #The WCS matrix is (hypothetically) the same for every image so the first header is fine
        self.filter_band = self.header_list[0]['filter'][0] #This assumes the filter is the same for every image in the set. We can make this a variable option later but for now idgaf.
                                                             #Another brief note: the last 0th index is because LCO lists filters as 'xp' with the p representing prime. We just want the actual band letter, not the p.


class Star:
    def __init__(self, icrs_coords, pixel_coords, x_min, x_max, SDSS_filter_band, source_id=None, instrumental_mag=None, instrumental_mag_err=None, SDSS_mag=None, SDSS_mag_err=None):
        self.icrs_coords = icrs_coords
        self.pixel_coords = pixel_coords
        self.x_min = x_min
        self.x_max = x_max
        self.SDSS_filter_band = SDSS_filter_band
        self.source_id = source_id
        self.instrumental_mag = instrumental_mag
        self.instrumental_mag_err = instrumental_mag_err

        SDSS_reference_search = SDSS.query_crossid(self.icrs_coords, fields=['ra', 'dec', f'psfMag_{SDSS_filter_band}', f'psfMagErr_{SDSS_filter_band}'], radius=15*u.arcsec, region=False) #narrow field cone search to find source based on ra, dec.
        if SDSS_reference_search:
            if SDSS_reference_search['type'] == 'STAR':
                self.SDSS_mag = SDSS_reference_search[f'psfMag_{SDSS_filter_band}']
                self.SDSS_mag_err = SDSS_reference_search[f'psfMagErr_{SDSS_filter_band}']

        self.pixel_radius = (self.x_max - self.x_min) / 2

In [63]:
files = sorted(glob.glob(r"C:\Users\Sam Whitebook\Documents\lubin_test_data\*"))

set1 = FrameSet(files)
set1.align_frames()
set1.create_template()
set1.store_aligned_frames()
set1.find_sources_from_template()
set1.create_reference_stars()

# plt.imshow(set1.template, norm=LogNorm(np.percentile(set1.template, 80), np.percentile(set1.template, 100)), cmap="gray", interpolation=None)

Set OBSGEO-B to    20.706958 from OBSGEO-[XYZ].
Set OBSGEO-H to     3033.997 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
